In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import re

### ID prop csv



In [2]:
# id_prop dataset
prop = "h"
# run_idx = 2
run_idx = "unlabel"
first_row = True
shuffle_seed = 11
shuffle = False
new_only = False
terminal_oxo = False
prop_map = {"oxo":"Oxo Formation Energy", "h": "Hydrogen Affinity Energy"}
# df_label = pd.read_csv("/scratch/yll6162/MOF-oxo/labels/Combined_Energies_Removed_Infeasible_Out.csv", index_col = 0)
# df_label = pd.read_csv("/scratch/yll6162/MOF-oxo/labels/Combined_Energies_Removed_Infeasible_Out_new_only.csv", index_col = 0)
id_filepath = f"/scratch/yll6162/MOF-oxo/MOFs_oms/id_prop_{prop}_{run_idx}"
if terminal_oxo:
    id_filepath += "_NN"
if first_row:
    id_filepath += "_first_row"
if new_only:
    id_filepath += "_new_only"
if shuffle:
    id_filepath += "_shuffled"

In [3]:
#Generate
folder_path = './mof_features'
df_fea = pd.read_csv(os.path.join(folder_path, 'Features_RACS.csv'), index_col=0)
df_fea_other = pd.read_csv(os.path.join(folder_path, 'RACSFeatures_other_MOFs.csv'), index_col=0)
df_fea_miss = pd.read_csv(os.path.join(folder_path, "RACSFeatures_missing.csv"), index_col=0)

df_fea = pd.concat([df_fea, df_fea_other, df_fea_miss])

if run_idx == "full":
    df_label = pd.read_csv("/scratch/yll6162/MOF-oxo/labels/id_prop_full.csv")
    df_label['sample'] = df_label["file"]
    df_label = df_label.merge(df_fea[['MOF Name','Metal_index']], how='inner', left_on=['id'], right_on = ['MOF Name'])
    df_label = df_label.rename(columns={"Metal_index":"Site"})
elif run_idx == "unlabel":
    df_label = pd.read_csv("/scratch/yll6162/MOF-oxo/labels/id_prop_unlabel.csv")

    # df_label['sample'] = df_label["file"]
    # df_label = df_label.merge(df_fea[['MOF Name','Metal_index']], how='inner', left_on=['id'], right_on = ['MOF Name'])
    # df_label = df_label.rename(columns={"Metal_index":"Site"})
else:
    # df_label = pd.read_csv("/scratch/yll6162/MOF-oxo/labels/Combined_Energies_Removed_Infeasible_Out_new_only.csv", index_col = 0)
    # df_label = pd.read_csv("/scratch/yll6162/MOF-oxo/labels/baremof_rem_inf_out.csv", index_col = 0)
    df_label = pd.read_csv("/scratch/yll6162/MOF-oxo/labels/baremofdf_rem_inf_and_NN.csv", index_col = 0)
    # df_label = pd.read_csv("/scratch/yll6162/MOF-oxo/labels/qmofruns_2_cleaned.csv", index_col = 0)


    # Filter by features dataset
    df_label = df_label.merge(df_fea[['MOF Name','Metal_index']], how='inner', left_on=['MOF', 'Site'], right_on = ['MOF Name','Metal_index'])
    if shuffle:
        df_label = df_label.sample(frac=1, random_state=shuffle_seed)
    if first_row:
        df_label = df_label[df_label.Metal.isin(['Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn'])]

    df_label['sample'] = df_label.MOF+".cif"

    df_label = df_label[["sample", "Site", prop_map[prop]]]

# df_label.to_csv(f"{id_filepath}.csv", index=None)
df_label

,id,file,MOF Name,Metal_index,Metal
0,qmof-0000295,qmof-0000295.cif,qmof-0000295,0,Cu
1,qmof-001ef83,qmof-001ef83.cif,qmof-001ef83,0,Cu
2,qmof-002580c,qmof-002580c.cif,qmof-002580c,0,Co
3,qmof-002a6da,qmof-002a6da.cif,qmof-002a6da,0,Cu
4,qmof-009b656,qmof-009b656.cif,qmof-009b656,0,Cu
...,...,...,...,...,...
2274,qmof-ffb8c94,qmof-ffb8c94.cif,qmof-ffb8c94,0,Cu
2275,qmof-ffc1922,qmof-ffc1922.cif,qmof-ffc1922,0,Cu
2276,qmof-ffc9870,qmof-ffc9870.cif,qmof-ffc9870,0,Cu
2277,qmof-ffccb82,qmof-ffccb82.cif,qmof-ffccb82,0,Cu


### Select RAC features


In [4]:
tl = False
one_hot = True
folder_path = './mof_features'
df_fea = pd.read_csv(os.path.join(folder_path, 'Features_RACS.csv'), index_col=0)
df_fea_other = pd.read_csv(os.path.join(folder_path, 'RACSFeatures_other_MOFs.csv'), index_col=0)
df_fea_miss = pd.read_csv(os.path.join(folder_path, "RACSFeatures_missing.csv"), index_col=0)

df_fea = pd.concat([df_fea, df_fea_other, df_fea_miss])
# df_fea.values
label_encoder = LabelEncoder()
# if 'compound possible' in df_fea.columns:
#     df_fea['compound possible'] = df_fea['compound possible'].replace({'False': 0, 'True': 1, False: 0, True: 1, '0.0': 0}).astype(float)
# for str_label in ['HOMO_character', 'HOMO_element', 'LUMO_character', 'LUMO_element']:
#     if str_label in df_fea.columns:
#         df_fea[str_label] = label_encoder.fit_transform(df_fea[str_label].astype(str))
if one_hot:
    encoder = OneHotEncoder(sparse=False)
else:
    encoder = label_encoder
encoded_feature = encoder.fit_transform(df_fea[['Metal']])
df_fea = df_fea.reset_index(drop=True)
if one_hot:
    df_fea = pd.concat([df_fea, pd.DataFrame(encoded_feature, columns=encoder.get_feature_names_out(['Metal']))], axis=1)
else:
    df_fea["encoded_Metal"] = encoded_feature
df_fea = df_fea.drop(columns = ['Metal'])
imputer = SimpleImputer(strategy='mean')
feature_cols = df_fea.columns[2:]
df_fea[feature_cols] = imputer.fit_transform(df_fea[feature_cols].values)
df_fea['sample'] = df_fea['MOF Name'] + '.cif'
filename = 'racs_all_clean.csv'
if not one_hot:
    filename = 'racs_all_clean_metal_one.csv'
# df_fea.to_csv(os.path.join(folder_path, filename))
# df_fea[['HOMO_character', 'HOMO_element', 'LUMO_character', 'LUMO_element']]


/scratch/yll6162/miniconda3/envs/mof/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/tmp/ipykernel_64289/1602736537.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fea['sample'] = df_fea['MOF Name'] + '.cif'


### Combine with ALIGNN Embeddings (Optional)


In [5]:
tl = True
# run = 'mof_form_e_embed'
# run = 'mof_ehull_embed'
# run = 'mof_bandgap_embed'
run = 'mof_dband_embed'
# embed_filepath = f"/scratch/yll6162/ALIGNNTL/examples/{run}/x+y+z/data0.csv"
if run_idx == "unlabel":
    embed_filepath = f"/data/yll6162/mof/{run}/x+y+z/unlabel_data0.csv"
else:
    embed_filepath = f"/data/yll6162/mof/{run}/x+y+z/data0.csv"
df_embed = pd.read_csv(embed_filepath)
df_embed = df_embed.drop_duplicates(subset = ['id'])
df_fea_all = df_fea.merge(df_embed, how='left', left_on = "MOF Name", right_on = "id").drop(columns = ['h',f'h_{run_idx}','oxo',f'oxo_{run_idx}','id','full'], errors='ignore')
df_fea = df_fea_all


### Feature Scale

In [6]:
from sklearn.preprocessing import StandardScaler

racs_col = [col for col in df_fea.columns if col.startswith("racs_")]
embed_cols = [str(i) for i in range(768)]
scaler_A = StandardScaler()
df_fea[racs_col] = scaler_A.fit_transform(df_fea[racs_col].values)
# if embed_cols:
#     scaler_B = StandardScaler()
#     df_fea[embed_cols] = scaler_B.fit_transform(df_fea[embed_cols].values)

### Generate Train Dataset

In [7]:
# prop = 'Oxo Formation Energy'
# assert prop in ['oxo','h']
if prop == "oxo":
    prop = 'Oxo Formation Energy'
elif prop == 'h':
    prop = 'Hydrogen Affinity Energy'

output_dir = './data/'

# df_qmof = pd.read_csv("./labels/qmofruns_2.csv", index_col = 0)
# df_qmof = pd.read_csv("/scratch/yll6162/MOF-oxo/labels/Combined_Energies_Removed_Infeasible_Out.csv", index_col = 0)
# qmof_filename = "/scratch/yll6162/MOF-oxo/labels/Combined_Energies_Removed_Infeasible_Out_new_only.csv"
# qmof_filename = "./labels/qmofruns_2.csv"
# qmof_filename = "/scratch/yll6162/MOF-oxo/labels/baremof_rem_inf_out.csv"
# if new_only:
#     assert "new_only" in qmof_filename
# else:
#     assert "new_only" not in qmof_filename
# df_qmof = pd.read_csv(qmof_filename, index_col = 0)

# if first_row:
#     df_qmof = df_qmof[df_qmof.Metal.isin(['Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn'])]
# df_qmof['sample'] = df_qmof.MOF + '.cif'

# df_qmof['prop'] = df_qmof[prop]
# df_qmof = df_qmof[['sample', 'Site', prop]]

df_qmof = df_label
df_fea['sample'] = df_fea['MOF Name'] + '.cif'
if run_idx == "full":
    
    df_qmof['sample'] = df_qmof['MOF Name'] + '.cif'
    df_sample = df_qmof.merge(df_fea, how='inner', left_on=['sample'], right_on = ['sample'])
    df_sample['ids'] = df_sample['sample'] + '_' + df_sample['Metal_index'].astype(int).astype(str)
    df_sample = df_sample.drop(columns = ['file', 'sample', 'MOF Name', 'Metal_index', "oxo_2"], errors="ignore")
    df_sample = df_sample.assign(**{'ids': df_sample.pop('ids')})
elif run_idx == "unlabel":
    df_qmof = df_qmof.drop(columns=["Metal", "id", "file"])
    df_qmof['sample'] = df_qmof['MOF Name'] + '.cif'
    df_sample = df_qmof.merge(df_fea.drop(columns=["MOF Name"]), how='inner', left_on=['sample', "Metal_index"], right_on = ['sample', 'Metal_index'])
    df_sample['ids'] = df_sample['sample'] + '_' + df_sample['Metal_index'].astype(int).astype(str)
    df_sample = df_sample.drop(columns = ['sample', 'MOF Name', 'Metal_index'])
    df_sample = df_sample.assign(**{'ids': df_sample.pop('ids')})
else:
    df_sample = df_qmof.merge(df_fea, how='inner', left_on=['sample', 'Site'], right_on = ['sample','Metal_index'])
    df_sample['ids'] = df_sample['sample'] + '_' + df_sample['Site'].astype(str)
    df_sample = df_sample.drop(columns = ['sample', 'Site', 'MOF Name', 'Metal_index'])
    df_sample = df_sample.assign(**{prop: df_sample.pop(prop)})
    df_sample = df_sample.assign(**{'ids': df_sample.pop('ids')})
if tl:
    filename = f"query_{run_idx}_{run}_{prop}"
    if terminal_oxo:
        filename += "_NN"
    if first_row:
        filename += "_first_row"
    if new_only:
        filename += "_new_only"

    if shuffle:
        filename += "_shuffled"
    df_sample.to_csv(os.path.join(output_dir, filename + ".csv"), index=False)
else:
    filename = f"query_{run_idx}_racs_{prop}"
    if terminal_oxo:
        filename += "_NN"
    if first_row:
        filename += "_first_row"
    if new_only:
        filename += "_new_only"

    if shuffle:
        filename += "_shuffled"
    df_sample.to_csv(os.path.join(output_dir, filename + ".csv"), index=False)
print(f"saved to {os.path.join(output_dir, filename + '.csv')}")

/tmp/ipykernel_64289/2527197982.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_sample['ids'] = df_sample['sample'] + '_' + df_sample['Metal_index'].astype(int).astype(str)


saved to ./data/query_unlabel_mof_dband_embed_Hydrogen Affinity Energy_first_row.csv


In [ ]:
racs_feature = ['racs_bb-linker_all_prop-I_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-I_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-I_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-I_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-I_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-I_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-T_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-T_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-T_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-T_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-T_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-T_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-X_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-X_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-X_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-X_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-X_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-X_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-covalent_radius_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-covalent_radius_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-covalent_radius_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-covalent_radius_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-covalent_radius_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-covalent_radius_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-z_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-z_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-z_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-z_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-z_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-z_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-I_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-I_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-I_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-I_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-I_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-I_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-T_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-T_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-T_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-T_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-T_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-T_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-X_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-X_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-X_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-X_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-X_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-X_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-covalent_radius_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-covalent_radius_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-covalent_radius_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-covalent_radius_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-covalent_radius_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-covalent_radius_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-z_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-z_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-z_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-z_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-z_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-z_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-I_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-I_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-I_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-I_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-I_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-I_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-T_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-T_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-T_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-T_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-T_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-T_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-X_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-X_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-X_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-X_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-X_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-X_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-covalent_radius_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-covalent_radius_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-covalent_radius_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-covalent_radius_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-covalent_radius_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-covalent_radius_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-z_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-z_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-z_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-z_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-z_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-z_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-I_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-I_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-I_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-I_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-I_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-I_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-T_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-T_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-T_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-T_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-T_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-T_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-X_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-X_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-X_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-X_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-X_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-X_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-covalent_radius_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-covalent_radius_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-covalent_radius_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-covalent_radius_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-covalent_radius_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-covalent_radius_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-z_scope-1_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-z_scope-1_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-z_scope-2_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-z_scope-2_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-z_scope-3_propagg-diff_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-z_scope-3_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-I_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-T_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-X_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-covalent_radius_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_all_prop-z_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-I_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-T_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-X_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-covalent_radius_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_connecting_prop-z_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-I_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-T_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-X_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-covalent_radius_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-linker_functional_prop-z_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-I_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-T_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-X_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-covalent_radius_scope-0_propagg-product_corragg-sum_bbagg-sum', 'racs_bb-nodes_prop-z_scope-0_propagg-product_corragg-sum_bbagg-sum']
df_fea_sample[df_fea_sample.duplicated(subset=racs_feature, keep=False)]

In [ ]:
list_1 = []
for feature in racs_feature:
    num_dup = len(df_fea_sample[df_fea_sample.duplicated(subset=[feature], keep=False)])
    if num_dup == 0:
        # print(feature)
        list_1.append(feature)
list_1